![IITIS](pictures/logoIITISduze.png)

# Introduction to Ising Model

The Ising model has begun its life as a model of ferromagnetism. Ernst Ising introduced it in the early 1920s, and it has since become a fundamental model for understanding phase transitions and emergent properties in various physical systems. In this course, we will focus on one problem related to the Ising model: finding its ground state (i.e., the state with the lowest energy) and a low energy spectrum. As will be shown later in the course, the Ising model is significant not only in physics but also influences fields such as finance, social dynamics, and optimization in computer science.




## Definition
Let us consider a vector of binary variables $|\bm{s}\rangle = (s_1, s_2, \ldots, s_N)$. These variables are called **spin** and can be in one of two states: up (+1) or down (-1), formally $s_i = \pm 1$. Spins are arranged on some simple graph $G$, which describes interactions between them. The edges of $G$ (denoted $E(G)$) determine which spin interacts with which. The strength of the interaction (or coupling) between $s_i$ and $s_j$ is denoted by $J_{ij} \in \mathbb{R}$. Additionally, each spin is subjected to the external magnetic field (or simply bias) $h_i \in \mathbb{R}$. The energy of the system is calculated in the following way:
$$
E(\bm{s}) = \sum_{(i, j) \in E(G)} J_{ij} s_i s_j + \sum_{i=1}^{N} h_i s_i
$$

PICTURE

From a computing standpoint, looking at the Ising model in terms of linear algebra may be easier. We look into the adjacency matrix of graph $G$ and treat each $J_{ij}$ as weight assigned to the edge. As $G$ is undirected, $J_{ij} = J_{ji}$. To avoid needles work, we can treat this matrix as an upper-triangular matrix and only care for $J_{ij}$ where $ i < j$. It is customary to denote this matrix by $J$. Additionally, we can order values $h_i$ into the vector $|\bm{h}\rangle$ Then the equation for $E(\bm{s})$ is equal:
$$
E(\bm{s}) = \langle\bm{s}|J|\bm{s}\rangle + \langle\bm{h}|\bm{s}\rangle
$$

### Exercise 1
Show that the two approaches mentioned above are indeed equal.

## Finding the ground state 

To find the ground state, one might initially think to compute the energies of every possible spin configuration and simply choose the one with the lowest energy. Let's consider a complete graph with three spins as an example.

In [10]:
import numpy as np
from itertools import product
from math import inf

# define Ising model (both ways)
h = {1: 0.5, 2: -1, 3: 0.75}
J = {(1, 2): 1, (1, 3): 0.5, (2, 3): -0.33}

h_vect = np.array([0.5, -1, 0.75])
J_matrix = np.array([[0, 1, 0.5],[0, 0, -0.33],[0, 0, 0]])

In [6]:
# Calulate direcly from first definition
best_energy = inf
best_state = {}
for configuration_vector in product([-1, 1], repeat=3):
    energy = 0
    configuration = {i: configuration_vector[i-1] for i in range(1, 4)}
    for (v1, v2), coupling_strength in J.items():
        energy += coupling_strength * configuration[v1] * configuration[v2]
    for v, bias in h.items():
        energy += bias * configuration[v]
    if energy < best_energy:
        best_energy = energy
        best_state = configuration

print(best_state)
print(best_energy)


{1: -1, 2: 1, 3: 1}
-2.58


In [12]:
# Calculate using vectors and matrices
best_energy = inf
best_state = []
for configuration_vector in product([-1, 1], repeat=3):
    configuration = np.array(configuration_vector)
    energy = np.dot(configuration.T, np.dot(J_matrix, configuration)) + np.dot(h_vect, configuration)
    if energy < best_energy:
        best_energy = energy
        best_state = configuration

print(best_state)
print(best_energy)

[-1  1  1]
-2.58


However, it becomes apparent that the number of operations required scales exponentially as $2^N$. This exponential growth quickly makes the task unmanageable. For instance, even with just 100 spins, this naive exhaustive search algorithm becomes practically impossible to execute.

Suprisingly, the exhaustive search method is the only known algorithm that **guarantees** finding the true ground state. 


## Hamiltonian

The Ising model hamiltonian may be formulated using Pauli matrices. As reminder, the Pauli $Z$ matrix $\sigma^{Z}$ is defined as:

$$\sigma^{Z} = \begin{bmatrix} 
1 & 0 \\ 
0 & -1 
\end{bmatrix}  $$

It is worth noting that $\sigma^{Z}$ has eigenvalues $-1$ and $1$ with coresponding eigenvectors $|0\rangle$ and $|1\rangle$ respectivelly. Additionaly, by telling that matrix $\sigma_i^{Z}$ acts on spin $i$ we mean: 

$$
\sigma_i^{Z} = \underbrace{I  \otimes \ldots \otimes I}_{\text{$i-1$~times}} \otimes \underbrace{\sigma^Z}_{\text{$i$~-th place}} \otimes \underbrace{ I \otimes \ldots \otimes I}_{\text{$N-i$~times}}
$$
